In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
len(sentence)

198

In [4]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {word: num for num, word in enumerate(world_set)}

In [5]:
print(vocab)

{'i': 0, 'u': 1, 'r': 2, '.': 3, 'e': 4, 'y': 5, 'B': 6, 'h': 7, 's': 8, 'f': 9, ',': 10, 'b': 11, "'": 12, 'a': 13, 'k': 14, 'm': 15, 'w': 16, ' ': 17, 'p': 18, 'd': 19, 'o': 20, 'c': 21, 'g': 22, 't': 23, 'l': 24, 'n': 25}


In [6]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [7]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [8]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i: i + sequence_length]
  y_str = sentence[i+1: i + sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [9]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[6, 2, 0, 21, 14, 17, 16, 13, 24, 24]
[2, 0, 21, 14, 17, 16, 13, 24, 24, 8]


In [10]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(vocab_size)[x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [12]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [11]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [13]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 2,  0, 21, 14, 17, 16, 13, 24, 24,  8])


In [18]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)
  
  def forward(self, x):
    x, state = self.rnn(x)
    x = self.fc(x)
    return x

In [19]:
net = Net(vocab_size, hidden_size, 2)

In [20]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [21]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [23]:
Y.shape

torch.Size([188, 10])

In [25]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

hhfahhhhhhahfhhhhhhhhhfffhfhfhfhaahfhfhahhhfhhphhaahhfhhhhhhhhhafhhhfffhhhhhhahfhhahahhfhhhhhfhfhhphfhhahhhhfhhhfhhfhhhfhhahhahhhhhahhhhafhhhfffhhhhhhahfhhhhhhhhhhfhfhhphhhhfhphffhhahhhhphhhhhfhaah
hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
hh hhhh hh  hhhhhhhhhhh hhhhhhhh hhhh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhh hhhhh hh  hhhhhhhhhhhhhhhhhhh hhh hhhhhh hhhhhhhhhhhhhhhh hhhhhhhhhhhhhh hhhhh hh  hhhhhhhhhhhhhh h hhh hh hh hhhhhhhhhhhhhhhh h
h                                                                                                                                                                                                    
                                                                                                                                                                                                     
          

In [26]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thingsm"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [224]:
# 다이너마이트 가사(I-I-I'm이나 morn'과 같은 단어들때문에 학습이 잘안되는 것같아 약간 수정!

sentence = ("Cause I'm in the stars tonight So watch me bring the fire and set the night alight"
            "Shoes on, get up in the morning Cup of milk, let's rock and roll"
            "King Kong, kick the drum, rolling on like a Rolling Stone")

In [279]:
sentence_word = sentence.split() # 1. split으로만 학습 -> 2. split하고 띄어쓰기 넣어주기

n=0
for i in sentence_word:
  sentence_word[n] = i + ' '
  n += 1

sentence_word

['Cause ',
 "I'm ",
 'in ',
 'the ',
 'stars ',
 'tonight ',
 'So ',
 'watch ',
 'me ',
 'bring ',
 'the ',
 'fire ',
 'and ',
 'set ',
 'the ',
 'night ',
 'alightShoes ',
 'on, ',
 'get ',
 'up ',
 'in ',
 'the ',
 'morning ',
 'Cup ',
 'of ',
 'milk, ',
 "let's ",
 'rock ',
 'and ',
 'rollKing ',
 'Kong, ',
 'kick ',
 'the ',
 'drum, ',
 'rolling ',
 'on ',
 'like ',
 'a ',
 'Rolling ',
 'Stone ']

In [280]:
len(sentence_word)

40

In [288]:
world_set = list(set(sentence_word))

vocab = {word:num for num, word in enumerate(world_set)}

# 1. 문자들 인코딩, 2. 단어별 인코딩 3. 단어 split + 띄어쓰기 첨가가

In [289]:
print(vocab)

{'Kong, ': 0, 'tonight ': 1, 'set ': 2, 'like ': 3, 'Rolling ': 4, 'and ': 5, 'stars ': 6, 'milk, ': 7, 'on ': 8, 'night ': 9, 'in ': 10, 'me ': 11, "let's ": 12, 'alightShoes ': 13, 'on, ': 14, 'So ': 15, 'bring ': 16, 'Stone ': 17, 'rollKing ': 18, 'morning ': 19, 'of ': 20, 'Cause ': 21, 'fire ': 22, 'rolling ': 23, 'get ': 24, 'rock ': 25, "I'm ": 26, 'kick ': 27, 'drum, ': 28, 'watch ': 29, 'up ': 30, 'the ': 31, 'a ': 32, 'Cup ': 33}


In [290]:
vocab_size = len(vocab)
print(vocab_size)

34


In [291]:
hidden_size = vocab_size
sequence_length = 3
learning_rate = 0.01

In [292]:
x_data = []
y_data = []

for i in range(0, len(sentence_word) - sequence_length):
  x_str = sentence_word[i : i + sequence_length]
  y_str = sentence_word[i+1 : i + sequence_length + 1]
  print(i, x_str, '->', y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 ['Cause ', "I'm ", 'in '] -> ["I'm ", 'in ', 'the ']
1 ["I'm ", 'in ', 'the '] -> ['in ', 'the ', 'stars ']
2 ['in ', 'the ', 'stars '] -> ['the ', 'stars ', 'tonight ']
3 ['the ', 'stars ', 'tonight '] -> ['stars ', 'tonight ', 'So ']
4 ['stars ', 'tonight ', 'So '] -> ['tonight ', 'So ', 'watch ']
5 ['tonight ', 'So ', 'watch '] -> ['So ', 'watch ', 'me ']
6 ['So ', 'watch ', 'me '] -> ['watch ', 'me ', 'bring ']
7 ['watch ', 'me ', 'bring '] -> ['me ', 'bring ', 'the ']
8 ['me ', 'bring ', 'the '] -> ['bring ', 'the ', 'fire ']
9 ['bring ', 'the ', 'fire '] -> ['the ', 'fire ', 'and ']
10 ['the ', 'fire ', 'and '] -> ['fire ', 'and ', 'set ']
11 ['fire ', 'and ', 'set '] -> ['and ', 'set ', 'the ']
12 ['and ', 'set ', 'the '] -> ['set ', 'the ', 'night ']
13 ['set ', 'the ', 'night '] -> ['the ', 'night ', 'alightShoes ']
14 ['the ', 'night ', 'alightShoes '] -> ['night ', 'alightShoes ', 'on, ']
15 ['night ', 'alightShoes ', 'on, '] -> ['alightShoes ', 'on, ', 'get ']
16 ['alight

In [293]:
print(x_data[0])
print(y_data[0])

[21, 26, 10]
[26, 10, 31]


In [294]:
x_one_hot = np.eye(vocab_size)[x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [295]:
print(X.shape)
print(Y.shape)

torch.Size([37, 3, 34])
torch.Size([37, 3])


In [296]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)
  
  def forward(self, x):
    x, state = self.rnn(x)
    x = self.fc(x)
    return x

In [297]:
net = Net(vocab_size, hidden_size, 2)

In [298]:
outputs = net(X)
print(outputs.shape)

torch.Size([37, 3, 34])


In [299]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), learning_rate)

In [300]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: 
            predict_str += ''.join([world_set[t] for t in result])
        else: 
            predict_str += world_set[result[-1]]

    print(predict_str)

get bring get get set get get get bring get set get get get set get bring set bring set get get set get set bring set get bring get get set get get get set get get bring 
the the the the the the the the the the the the get the the the the the the the the the the watch the the the the the the the the the the the the the the the 
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
the the the the the the the the the the th

In [207]:
predict_str

# sequence_length가 10일때, layer 2층, 문자들 인코딩

"unse I'm in the mtars tonight So watch me bring the fire and set the night alightShoes on, get up in the morning Cup of milk, let's rock and rollKing Kong, kick the drum, rolling on like a Rolling Stoni"

In [191]:
predict_str

# sequence_length가 15일때, layer 2층 -> 10보다 나은거 같다, 문자들 인코딩

"uuse I'm in the stars tonight So watch me bring the fire and set the night alightShoes on, get up in the morning Cup of milk, let's rock and rollKing Kong, kick the drum, rolling on like a Rolling Stone"

In [223]:
predict_str

# sequence_length가 20일때, layer 2층, 문자들 인코딩

"upm  S'm in the stars tonight So watch me bring the fire and set the night alightShoes on, get up in the morning Cup of milk, let's rock and rollKing Kong, kick the drum, rolling on like a Rolling Ston "

In [240]:
predict_str

# sequence_length가 10일때, layer 1층, 문자들 인코딩

"uuse I'm in the stars tonight So watch me bring the fire and set the night alightShoes on, get up in the sorning oup of milk, let's rock and rollKing Kong, kick the drum, rolling on like a Rolling Stone"

In [251]:
predict_str

# sequence_length가 15일때, layer 1층, 문자들 인코딩

"u mt m'm in the stars tonight So watch me bring the fire and set the night alightShoes on, get up in the morning Cup of milk, let's rock and rollKing Kong, kick the drum, rolling on like a Rolling oo n "

In [270]:
predict_str

# sequence_length가 3일때, layer 1층, 단어들 인코딩
# 단어들이 잘 학습은 되었는데 띄어쓰기가 반영이 안됨...

"I'minthestarstonightSowatchmebringthefireandsetthenightalightShoeson,getupinthemorningCupofmilk,let'srockandrollKingKong,kickthedrum,rollingonlikeaRollingStone"

In [301]:
predict_str

# sequence_length가 3일때, layer 1층, 단어들 인코딩 + 띄어쓰기
# 구두점이 문장에 없어서 반영이 안된것 같다
# 단어별로 학습하는 것이 성능이 제일 좋다!

"I'm in the stars tonight So watch me bring the fire and set the night alightShoes on, get up in the morning Cup of milk, let's rock and rollKing Kong, kick the drum, rolling on like a Rolling Stone "